# --Under Development--

## **Notebook for gene target predictions to increase TAG production**

By Garrett Roell and Christina Schenk

Tested on biodesign_3.7 kernel on jprime

This notebook predicts interventions to increase TAG production for R. opacus cultures growing with phenol and glucose.

**Import python packages**

In [10]:
import pandas as pd
import cobra
import cplex
import sys
sys.path.append('../src')
from optforce import optforce

**Load model**

In [11]:
file_name =  '../models/r_pacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

In [12]:
model

Name,ropacus_annotated_curated
Memory address,0x07fb668e35a10
Number of metabolites,1956
Number of reactions,3025
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


**Define constants**

In [ ]:
glucose_uptake_rate = 3.582471 # mmol glucose / g dry cell weight / hr
phenol_uptake_rate = 1.352072 # mmol phenol / g dry cell weight / hr

# glucose_growth_rate = 0.260195 # hr-1
# phenol_growth_rate = 0.065488 # hr-1

**Define peripheral reactions**

In [4]:
# get central carbon reaction ids from 13C data mapping file
file_name =  '../data/central_fluxes/glucose_13C.csv'
central_flux_data = pd.read_csv(file_name)
central_reactions_list = list(central_flux_data['Forward Reactions'].values)

# filter out reactions that do not have a mapping to the genome scale model
central_reactions_list = [reactions for reactions in central_reactions_list if str(reactions) != 'nan']

# remove 'reverse' from all reactions
central_reactions_list = [reactions.replace('reverse_', '') for reactions in central_reactions_list]

# remove parentheses from all reactions
central_reactions_list = [reactions.replace('(', '').replace(')', '') for reactions in central_reactions_list]

# get the list of central reactions
central_reactions = []
for reactions in central_reactions_list:
    # remove 'or' from all reactions
    reaction_list = reactions.split(' or ')
    
    # remove 'and' from all reactions
    for reaction in reaction_list:
        if ' and ' in reaction:
            split_reactions = reaction.split(' and ')
            for rxn in split_reactions:
                central_reactions.append(rxn.strip())
        else:
            central_reactions.append(reaction)

# get the list of peripheral reactions
peripheral_reactions = []
for rxn in model.reactions:
    if rxn.id not in central_reactions:
        peripheral_reactions.append(rxn.id)
        
print(f' There are {len(peripheral_reactions)} peripheral reactions')

 There are 2924 peripheral reactions


**TAG production from glucose with 0 knocked out genes**

In [5]:
with model:
    # set up medium
    medium = model.medium
    medium["EX_glc__D_e"] = 100. #sub_uptake_rate
    medium["EX_phenol_e"] = 0. #sub_uptake_rate
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    
    # set up biomass reaction
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
    model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
    
    # run FBA
    model.objective = 'Growth_Glucose'
    glucose_fba_sol = model.optimize()
    tag_production = glucose_fba_sol.fluxes['tag_production']
    
    print(f'Original TAG Production: {tag_production}')

EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_glc__D_e 100.0
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_mobd_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
Original TAG Production: 0.0


**TAG production from glucose with 1 knocked out gene**

In [6]:
with model:
    # set up medium
    medium = model.medium
    medium["EX_glc__D_e"] = 100. #sub_uptake_rate
    medium["EX_phenol_e"] = 0. #sub_uptake_rate
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    
    # set up biomass reaction
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
    model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
    biomass = 'Growth_Glucose' # not sure what this does
    
    # setup optforce
    model.objective = 'Growth_Glucose'
    target = 'tag_production'
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
    model.solver = 'cplex'
    one_knockout_solution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction, 
            peripheral_reactions, high_order_must, number_of_solutions, time_limit)#, verbose=False)
    
    display(one_knockout_solution)

EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_glc__D_e 100.0
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_mobd_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
excluded reactions
['12DGR140tipp', '13PPDH', '1P2CBXLCYCL', '1P2CBXLR', '23CTI1', '23CTI2', '23CTI3', '24DECOAR', '25DKGLCNt2rpp', '25DKGLCNtex', '26DAHtex', '2AGPA120tipp', '2AGPA141tipp', '2AGPA160tipp', '2AGPA161tipp', '2AGPA181tipp', '2AGPE120tipp', '2AGPE141tipp', '2AGPE160tipp', '2AGPE161tipp', '2AGPE180tipp', '2AGPE181tipp', '2AGPEAT120', '2AGPEAT141', '2AGPEAT160', '2AGPEAT161', '2AGPEAT180', '2AGPEAT181', '2AGPG120tipp', '2AGPG140tipp', '2AGPG141tipp', '2AGPG160tipp', '2AGPG161tipp', '2AGPG180tipp', '2AGPG181tipp', '2AGPGAT120', '2AGPGAT140', '2AGPGAT141', '2AGPGAT160', '2AGPGAT161', '2AGPGAT180', '2AGPGAT181', '2DDARAA', '2DGULRx', '2DHGLCK', '2DHGLCNkt_tpp', '2DHGLCNtex', '2HH24DDH', '2HH24D

{'Must_solution': {'MustL': [],
  'MustU': [],
  'MustLL': [],
  'MustUU': [],
  'MustLU': []},
 'Force_solution': []}

**TAG production from glucose with 2 knocked out genes**

In [9]:
with model:
    # set up medium
    medium = model.medium
    medium["EX_glc__D_e"] = 100. #sub_uptake_rate
    medium["EX_phenol_e"] = 0. #sub_uptake_rate
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    
    # set up biomass reaction
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
    model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
    
    
    # setup optforce
    model.objective = 'Growth_Glucose'
    biomass = 'Growth_Glucose'
    target = 'tag_production'
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
    model.solver = 'cplex'
    two_knockout_solution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction, peripheral_reactions,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
    
    display(two_knockout_solution)

EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_glc__D_e 100.0
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
EX_mobd_e 1000.0
excluded reactions
['12DGR140tipp', '13PPDH', '1P2CBXLCYCL', '1P2CBXLR', '23CTI1', '23CTI2', '23CTI3', '24DECOAR', '25DKGLCNt2rpp', '25DKGLCNtex', '26DAHtex', '2AGPA120tipp', '2AGPA141tipp', '2AGPA160tipp', '2AGPA161tipp', '2AGPA181tipp', '2AGPE120tipp', '2AGPE141tipp', '2AGPE160tipp', '2AGPE161tipp', '2AGPE180tipp', '2AGPE181tipp', '2AGPEAT120', '2AGPEAT141', '2AGPEAT160', '2AGPEAT161', '2AGPEAT180', '2AGPEAT181', '2AGPG120tipp', '2AGPG140tipp', '2AGPG141tipp', '2AGPG160tipp', '2AGPG161tipp', '2AGPG180tipp', '2AGPG181tipp', '2AGPGAT120', '2AGPGAT140', '2AGPGAT141', '2AGPGAT160', '2AGPGAT161', '2AGPGAT180', '2AGPGAT181', '2DDARAA', '2DGULRx', '2DHGLCK', '2DHGLCNkt_tpp', '2DHGLCNtex', '2HH24DDH', '2HH24D

{'Must_solution': {'MustL': [],
  'MustU': [],
  'MustLL': [],
  'MustUU': [],
  'MustLU': []},
 'Force_solution': []}

**Define a function to take in a solution and output biomass and TAG flux**

In [18]:
def optforce_solution_to_fluxes(solution, substrate):
    
    # setup media and objective function
    with model:
        medium = model.medium
        if substrate == 'glucose':
            medium["EX_glc__D_e"] = glucose_uptake_rate
            medium["EX_phenol_e"] = 0
            model.objective = 'Growth_Glucose'
        elif substrate == 'phenol':
            medium["EX_glc__D_e"] = 0
            medium["EX_phenol_e"] = phenol_uptake_rate
            model.objective = 'Growth_Phenol'
        model.medium = medium
        
        # check medium
        {print(k, v) for k, v in model.medium.items()}
        
        # get the reference flux solution
        print('getting reference solution')
        max_biomass_sol = model.optimize()
        print(max_biomass_sol)
        
        
        # setup knockdowns
        for reaction_id in solution:
            print()
            print(f'knocking down {reaction_id} which has {max_biomass_sol[reaction_id]} flux in WT solution')
            model.reactions.get_by_id(reaction_id).upper_bound = 0.5 * glucose_fba_sol.fluxes[reaction_id]
            model.reactions.get_by_id(reaction_id).lower_bound = 0.5 * glucose_fba_sol.fluxes[reaction_id]
            
            moma_solution = cobra.flux_analysis.moma(model, max_biomass_sol)

            print(moma_solution)
            
        
        # run FBA for maximum biomass
#         max_biomass_sol = model.optimize()
#         max_biomass_biomass_production = max_biomass_sol.objective_value
#         max_biomass_tag_production = max_biomass_sol.fluxes['tag_production']
        
#         # run FBA for maximum TAG production
#         model.objective = 'tag_production'
#         max_tag_sol = model.optimize()
#         max_tag_biomass_production = max_tag_sol.fluxes['Growth_Glucose']
#         max_tag_tag_production = max_tag_sol.fluxes['tag_production']
        
#         print(f'max_biomass_biomass_production: {max_biomass_biomass_production}')
#         print(f'max_biomass_tag_production: {max_biomass_tag_production}')
#         print(f'max_tag_biomass_production: {max_tag_biomass_production}')
#         print(f'max_tag_tag_production: {max_tag_tag_production}')
#         print()

        
        

**Nicely display glucose optforce results**

In [23]:
solutions = [
    [], # wild type
    ['FACOAL150_anteiso'], 
    ['KAS4'], 
    ['Clt'], 
    ['FACOAL140_ISO', 'KAS1'], 
    ['FACOAL140_ISO', 'GCALDD'], 
    ['FACOAL140_ISO', 'RBFSa']
]

glucose_optforce_data = []
solution_number = 0

# solution can be an empty array (WT), an array with one knockout, or an array with many knockouts
for solution in solutions:
    # handle wildtype case
    if len(solution) == 0:
        optforce_solution_to_fluxes(solution, 'glucose')
        glucose_optforce_data.append({
            'Solution': chr(65 + solution_number), # convert the index to a letter (0 = A, 1 = B, ...)
            'ID': 'Wild',
            'Knockouts': 'Complete Network',
            'Enzyme': '',
            'Biomass Flux (1/hr)': '',
            'TAG Flux (mmol/hr)': '',
        })
    # handle knockout case
    else: 
        for index, reaction_id in enumerate(solution):
            if index == 0:
                optforce_solution_to_fluxes(solution, 'glucose')
                glucose_optforce_data.append({
                    'Solution': chr(65 + solution_number), # convert the number to a letter (0 = A, 1 = B, ...)
                    'ID': reaction_id,
                    'Knockouts': model.reactions.get_by_id(reaction_id).reaction,
                    'Enzyme': model.reactions.get_by_id(reaction_id).name,
                    'Biomass Flux (1/hr)': '',
                    'TAG Flux (mmol/hr)': '',
                })
            else:
                glucose_optforce_data.append({
                    'Solution': '',
                    'ID': reaction_id,
                    'Knockouts': model.reactions.get_by_id(reaction_id).reaction,
                    'Enzyme': model.reactions.get_by_id(reaction_id).name,
                    'Biomass Flux (1/hr)': '',
                    'TAG Flux (mmol/hr)': '',
                })
    solution_number += 1
        
    
glucose_df = pd.DataFrame(glucose_optforce_data)
glucose_df
# display(glucose_df.to_string(index=False))



EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_glc__D_e 3.582471
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_mobd_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
getting reference solution
<Solution 0.481 at 0x7f8b406b0c10>
EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_glc__D_e 3.582471
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_mobd_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
getting reference solution
<Solution 0.481 at 0x7f8b406b9bd0>

knocking down FACOAL150_anteiso which has 0.014720295861838617 flux in WT solution
<Solution infeasible at 0x7f8b3dcb41d0>
EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_glc__D_e 3.582471
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2

,Solution,ID,Knockouts,Enzyme,Biomass Flux (1/hr),TAG Flux (mmol/hr)
0,A,Wild,Complete Network,,,
1,B,FACOAL150_anteiso,atp_c + coa_c + fa4_c <=> amp_c + fa4coa_c + p...,Fatty acid CoA ligase anteiso C150,,
2,C,KAS4,2mbcoa_c + 14.0 h_c + 5.0 malcoa_c + 10.0 nadp...,B-ketoacyl synthetase (Anteiso-C15:0),,
3,D,Clt,cl_e <=> cl_c,Major Facilitator(MFS) TCDB:2.A.1.14.6,,
4,E,FACOAL140_ISO,atp_c + coa_c + fa1_c <=> amp_c + fa1coa_c + p...,Fatty acid CoA ligase Iso C140,,
5,,KAS1,14.0 h_c + ibcoa_c + 5.0 malcoa_c + 10.0 nadph...,B-ketoacyl synthetase (Iso-C14:0),,
6,F,FACOAL140_ISO,atp_c + coa_c + fa1_c <=> amp_c + fa1coa_c + p...,Fatty acid CoA ligase Iso C140,,
7,,GCALDD,gcald_c + h2o_c + nad_c --> glyclt_c + 2.0 h_c...,Glycolaldehyde dehydrogenase,,
8,G,FACOAL140_ISO,atp_c + coa_c + fa1_c <=> amp_c + fa1coa_c + p...,Fatty acid CoA ligase Iso C140,,
9,,RBFSa,4r5au_c + db4p_c --> dmlz_c + 2.0 h2o_c + pi_c,Riboflavin synthase,,


In [11]:
model.metabolites.get_by_id('cl_e')

Metabolite identifier,cl_e
Name,Chloride
Memory address,0x07f8b693561d0
Formula,Cl
Compartment,C_e
In 2 reaction(s),"Clt, EX_cl_e"


**TAG production from phenol with 0 knocked out genes**

In [6]:
with model:
    # set up medium
    medium = model.medium
    medium["EX_phenol_e"] = 100. #sub_uptake_rate
    medium["EX_glc__D_e"] = 0.
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    
    # set up biomass reaction
    model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
    model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    
    # run FBA
    model.objective = 'Growth_Phenol'
    fbasol = model.optimize()
    tag_production = fbasol.fluxes['tag_production']
    
    print(f'Original TAG Production: {tag_production}')

EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
EX_mobd_e 1000.0
EX_phenol_e 100.0
Original TAG Production: 0.0


**TAG production from phenol with 1 knocked out gene**

In [7]:
with model:
    # set up medium
    medium = model.medium
    medium["EX_phenol_e"] = 100. #sub_uptake_rate
    medium["EX_glc__D_e"] = 0.
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    
    # set up biomass reaction
    model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
    model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    
    # setup optforce
    target = 'tag_production'
    model.objective = 'Growth_Phenol'
    biomass = 'Growth_Phenol'
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 # 0.1 Paper
    minimum_target_fraction = 0.75 # 0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180 # 3 minutes
    model.solver = 'cplex'
    one_knockout_solution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
    
    display(one_knockout_solution)

EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
EX_mobd_e 1000.0
EX_phenol_e 100.0
Inside add_force function. The value of maximum_knockout is 2


{'Must_solution': {'MustL': ['KAS4',
   'FACOAL150_anteiso',
   'FACOAL150_ISO',
   'KAS3',
   'FACOAL170_anteiso',
   'KAS12',
   'Clt',
   'FACOAL170_ISO',
   'KAS11',
   'FACOAL160_ISO',
   'KAS6'],
  'MustU': [],
  'MustLL': [['GCALDD', 'FACOAL180_2'],
   ['RBFSa', 'FACOAL180_2'],
   ['DB4PS', 'FACOAL180_2'],
   ['DB4PS', 'GCALDD'],
   ['RBFSa', 'GCALDD'],
   ['RBFSa', 'DB4PS'],
   ['DHFS', 'FACOAL180_2'],
   ['ADCL', 'FACOAL180_2'],
   ['ADCS', 'FACOAL180_2'],
   ['FACOAL180_2', 'GTPCI'],
   ['DHFS', 'GCALDD'],
   ['ADCL', 'GCALDD'],
   ['ADCS', 'GCALDD'],
   ['GCALDD', 'GTPCI'],
   ['RBFSa', 'DHFS'],
   ['DHFS', 'DB4PS'],
   ['RBFSa', 'GTPCI'],
   ['RBFSa', 'ADCS'],
   ['DB4PS', 'ADCS'],
   ['DB4PS', 'GTPCI'],
   ['RBFSa', 'ADCL'],
   ['ADCL', 'DB4PS'],
   ['FACOAL180_2', 'PANTS'],
   ['PNTK', 'FACOAL180_2'],
   ['FACOAL180_2', 'DPR'],
   ['MOHMT', 'FACOAL180_2'],
   ['PPCDC', 'FACOAL180_2'],
   ['PPNCL3', 'FACOAL180_2'],
   ['MOHMT', 'DB4PS', 'MECDPS'],
   ['PPNCL3', 'GCALDD'],


**TAG production from phenol with 2 knocked out genes**

In [8]:
with model:
    # set up medium
    medium = model.medium
    medium["EX_phenol_e"] = 100. #sub_uptake_rate
    medium["EX_glc__D_e"] = 0.
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    
    # set up biomass reaction
    model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
    model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    
    # setup optforce
    target = 'tag_production'
    model.objective = 'Growth_Phenol'
    biomass = 'Growth_Phenol'
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
    model.solver = 'cplex'
    two_knockout_solution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
    
    display(two_knockout_solution)

EX_ca2_e 1000.0
EX_cl_e 1000.0
EX_cobalt2_e 1000.0
EX_cu2_e 1000.0
EX_fe2_e 1000.0
EX_fe3_e 1000.0
EX_h2o_e 1000.0
EX_h_e 1000.0
EX_k_e 1000.0
EX_mg2_e 1000.0
EX_mn2_e 1000.0
EX_nh4_e 1000.0
EX_o2_e 1000.0
EX_pi_e 1000.0
EX_so4_e 1000.0
EX_zn2_e 1000.0
EX_phenol_e 100.0
EX_mobd_e 1000.0
Inside add_force function. The value of maximum_knockout is 2


{'Must_solution': {'MustL': ['FACOAL150_anteiso',
   'KAS4',
   'FACOAL150_ISO',
   'KAS3',
   'KAS12',
   'FACOAL170_anteiso',
   'Clt',
   'KAS11',
   'FACOAL170_ISO',
   'FACOAL160_ISO',
   'KAS6'],
  'MustU': [],
  'MustLL': [['FACOAL180_2', 'GCALDD'],
   ['FACOAL180_2', 'DB4PS'],
   ['FACOAL180_2', 'RBFSa'],
   ['GCALDD', 'RBFSa'],
   ['GCALDD', 'DB4PS'],
   ['DB4PS', 'RBFSa'],
   ['FACOAL180_2', 'DHFS'],
   ['FACOAL180_2', 'ADCL'],
   ['FACOAL180_2', 'ADCS'],
   ['FACOAL180_2', 'GTPCI'],
   ['GCALDD', 'ADCL'],
   ['GCALDD', 'DHFS'],
   ['GTPCI', 'RBFSa'],
   ['GCALDD', 'ADCS'],
   ['GCALDD', 'GTPCI'],
   ['DHFS', 'RBFSa'],
   ['ADCL', 'RBFSa'],
   ['DB4PS', 'GTPCI'],
   ['ADCS', 'RBFSa'],
   ['DHFS', 'DB4PS'],
   ['ADCL', 'DB4PS'],
   ['ADCS', 'DB4PS'],
   ['FACOAL180_2', 'PNTK'],
   ['FACOAL180_2', 'PPNCL3'],
   ['FACOAL180_2', 'PPCDC'],
   ['FACOAL180_2', 'PANTS'],
   ['FACOAL180_2', 'DPR'],
   ['FACOAL180_2', 'MOHMT'],
   ['GCALDD', 'PPNCL3'],
   ['GCALDD', 'PPCDC'],
   ['GCAL

# Old implementation

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
#     model.objective = 'Growth_Glucose'
#     fbasol = model.optimize()
#     TAGprod = fbasol.fluxes['tag_production']
#     display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
with model:
    medium = model.medium
    
    model.reactions.get_by_id('EX_tag').lower_bound = 0 # do not allow TAG to enter the cell
    
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
#         model.reactions.get_by_id('Growth').upper_bound = 0
#         model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    {print(k, v) for k,v in model.medium.items()}
    model.objective = 'Growth_Glucose'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    
    
    display('Original TAG Production', TAGprod)
    display('*********************')

### **1 Knockout Glucose**

In [ ]:
display(oursolution)

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
#     model.objective = 'Growth_Glucose'
#     fbasol = model.optimize()
#     TAGprod = fbasol.fluxes['tag_production']
#     display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'EX_tag'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
display(oursolution)

### **1 Knockout Phenol**

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='phenol'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective='Growth_Phenol'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
display(oursolution)

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='phenol'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective='Growth_Phenol'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'EX_tag'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
display(oursolution)

### **2 Knockouts Glucose**

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
#     model.objective = 'Growth_Glucose'
#     fbasol = model.optimize()
#     TAGprod = fbasol.fluxes['tag_production']
#     display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
display(oursolution)

### **2 Knockouts Phenol**

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='phenol'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective='Growth_Phenol'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
display(oursolution)